# Notes

- improve using this [colab](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/keras/Keras_pipeline_with_Weights_and_Biases.ipynb#scrollTo=-303Shsata7Z)

# Run this and restart runtime

In [27]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [28]:
%%bash
pip uninstall tensorflow -y
pip install tf-nightly==2.10.0.dev20220427

pip uninstall keras -y
pip install keras-nightly==2.10.0.dev2022042807

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
!nvidia-smi

Fri Jul 15 22:08:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    30W /  70W |  14628MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
!rm -rf Whats-this-rock/
!git clone https://github.com/udaylunawat/Whats-this-rock.git

Cloning into 'Whats-this-rock'...
remote: Enumerating objects: 515, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 515 (delta 33), reused 46 (delta 18), pack-reused 446
Receiving objects: 100% (515/515), 626.38 KiB | 15.66 MiB/s, done.
Resolving deltas: 100% (300/300), done.


In [31]:
%cd /content/Whats-this-rock/
!pip install -r requirements-dev.txt

/content/Whats-this-rock
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
try:
    import keras_cv
except:
    print("Install Tensorflow >= 2.9 and Restart runtime.")

# Setup

### Uploading Kaggle.json

In [1]:
%cd /content/Whats-this-rock/

/content/Whats-this-rock


In [3]:
from google.colab import files
print("Upload Kaggle.json")
files.upload();

Upload Kaggle.json


Saving kaggle.json to kaggle (1).json


In [4]:
!sh setup.sh

 93% 324M/349M [00:02<00:00, 135MB/s]
100% 349M/349M [00:02<00:00, 132MB/s]


In [ ]:
!python preprocess.py --root data/1_extracted/Rock_Dataset/ \
                      --remove_class minerals
                    #   --undersample \
# data/1_extracted/Rock_Dataset/sedimentary\ rocks

### Undersampling using pandas

In [ ]:
# import pandas as pd
# import os
# balanced_df = pd.read_csv(os.path.join('data/3_consume/', 'balanced_image_paths.csv'), index_col = 0)
# balanced_df['classes'].value_counts()

In [ ]:
# all_df = pd.read_csv(os.path.join('data/3_consume/', 'all_image_paths.csv'), index_col = 0)
# all_df['classes'].value_counts()

In [ ]:
# # https://stackoverflow.com/a/44546263/9292995
# drop_df = pd.merge(all_df, balanced_df, on=['image_path','classes'], how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)
# drop_df['image_path'].apply(lambda x:os.remove(x))

In [ ]:
# balanced_df

# Run pipeline in notebook

In [6]:
# import json
# class dotdict(dict):
#     """dot.notation access to dictionary attributes"""
#     __getattr__ = dict.get
#     __setattr__ = dict.__setitem__
#     __delattr__ = dict.__delitem__


# with open('config.json') as f:
#     config = dotdict(json.load(f))

In [ ]:
config = dict(
    root_dir = "data/4_tfds_dataset",
    project_name = "Whats-this-rock",
    model_name = "resnet50",
    sample_size = 1.0,
    augment= True,
    optimizer= "Adam",
    learning_rate = 0.0005,
    batch_size = 64,
    epochs = 5,
    image_size = 224,
    trainable = False,
    lr_decay_rate = 0.7,
    loss_fn = "CategoricalCrossentropy(label_smoothing=0.1)",
    metrics = ['acc'],
    earlystopping_patience = 5,
    notes= "keras-cv augment run"
)

In [7]:
print(config)

{'root_dir': 'data/1_extracted/Rock_Dataset/', 'project_name': 'rock_classification', 'model_name': 'efficientnet', 'sample_size': 1.0, 'augment': 'False', 'optimizer': 'Adam', 'learning_rate': 0.01, 'batch_size': 64, 'epochs': 20, 'image_size': 224, 'trainable': False}


In [8]:
# from data_utilities import get_data
# from model_utilities import get_generators

# train_df, test_df = get_data(config.sample_size)

# train_generator, val_generator, test_generator = get_generators(
#     config, train_df, test_df)

# num_classes = len(train_generator.class_indices)

## Using keras_cv

In [9]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras import losses
from tensorflow.keras import optimizers
import keras_cv

AUTOTUNE = tf.data.AUTOTUNE

In [10]:
!rm -rf data/4_tfds_dataset
!mkdir data/4_tfds_dataset checkpoints

# https://stackoverflow.com/a/64006242/9292995
import splitfolders

# If your datasets is balanced (each class has the same number of samples), choose ratio otherwise fixed.
splitfolders.fixed('data/2_processed', output="data/4_tfds_dataset", oversample=True,
                   seed=1337) 

Copying files: 4537 files [00:00, 7701.75 files/s]
Oversampling: 3 classes [00:00,  7.50 classes/s]


In [11]:
# https://www.tensorflow.org/datasets/api_docs/python/tfds/folder_dataset/ImageFolder
builder = tfds.ImageFolder('data/4_tfds_dataset')
print(builder.info)  # num examples, labels... are automatically calculated
data = builder.as_dataset(split=None, as_supervised=True)

tfds.core.DatasetInfo(
    name='image_folder',
    version=1.0.0,
    description='Generic image classification dataset.',
    homepage='https://www.tensorflow.org/datasets/catalog/image_folder',
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    total_num_examples=6741,
    splits={
        'test': 300,
        'train': 6141,
        'val': 300,
    },
    supervised_keys=('image', 'label'),
    citation="""""",
    redistribution_info=,
)



In [12]:
num_classes = builder.info.features['label'].num_classes
config["num_classes"] = num_classes

In [13]:
# # OPTIONAL show dataset
# ds = builder.as_dataset(split='train', shuffle_files=True)
# tfds.show_examples(ds, builder.info)

In [14]:
# # https://stackoverflow.com/a/37343690/9292995
# # https://keras.io/guides/keras_cv/cut_mix_mix_up_and_rand_augment/

IMAGE_SIZE = (config["image_size"], config["image_size"])


def to_dict(image, label):
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32)
    label = tf.one_hot(label, config["num_classes"])
    return {"images": image, "labels": label}


def prepare_dataset(dataset, split):
    if split == "train":
        return (
            dataset.shuffle(10 * config["batch_size"])
            .map(to_dict, num_parallel_calls=AUTOTUNE)
            .batch(config["batch_size"])
        )
    elif split == "val" or split == "test":
        return (
            dataset.map(to_dict, num_parallel_calls=AUTOTUNE)
            .batch(config["batch_size"])
        )


def load_dataset(split="train"):
    dataset = data[split]
    return prepare_dataset(dataset, split)


train_dataset = load_dataset()

In [15]:
def visualize_dataset(dataset, title):
    plt.figure(figsize=(12, 12)).suptitle(title, fontsize=18)
    for i, samples in enumerate(iter(dataset.take(9))):
        images = samples["images"]
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[0].numpy().astype("uint8"))
        plt.axis("off")
    plt.show()

def apply_rand_augment(inputs):
    inputs["images"] = rand_augment(inputs["images"])
    return inputs

rand_augment = keras_cv.layers.RandAugment(
    value_range=(0, 255),
    augmentations_per_image=3,
    magnitude=0.3,
    magnitude_stddev=0.2,
    rate=0.5,
)

cut_mix = keras_cv.layers.CutMix()
mix_up = keras_cv.layers.MixUp()


def cut_mix_and_mix_up(samples):
    samples = cut_mix(samples, training=True)
    samples = mix_up(samples, training=True)
    return samples

def preprocess_for_model(inputs):
    images, labels = inputs["images"], inputs["labels"]
    images = tf.cast(images, tf.float32)
    return images, labels

In [16]:
# visualize_dataset(train_dataset, title="Before Augmentation")

In [17]:
# train_dataset = load_dataset().map(apply_rand_augment, num_parallel_calls=AUTOTUNE)
# visualize_dataset(train_dataset, title="After RandAugment")

In [18]:
# train_dataset = load_dataset().map(cut_mix_and_mix_up, num_parallel_calls=AUTOTUNE)
# visualize_dataset(train_dataset, title="After cut_mix and mix_up")

In [19]:
train_dataset = (
    load_dataset()
    .map(apply_rand_augment, num_parallel_calls=AUTOTUNE)
    .map(cut_mix_and_mix_up, num_parallel_calls=AUTOTUNE)
)

# visualize_dataset(train_dataset, "CutMix, MixUp and RandAugment")

train_dataset = train_dataset.map(preprocess_for_model, num_parallel_calls=AUTOTUNE)

val_dataset = load_dataset(split="val")
val_dataset = val_dataset.map(preprocess_for_model, num_parallel_calls=AUTOTUNE)

test_dataset = load_dataset(split="test")
test_dataset = test_dataset.map(preprocess_for_model, num_parallel_calls=AUTOTUNE)

In [51]:
input_shape = (config["image_size"], config["image_size"]) + (3,)

def get_model():
    input_t = keras.Input(shape=(config["image_size"], config["image_size"], 3))
    res_model = keras.applications.ResNet50(include_top=False,
                                            weights="imagenet",
                                            input_tensor=input_t)

    to_res = IMAGE_SIZE

    model = keras.models.Sequential()
    model.add(keras.layers.Lambda(lambda image: tf.image.resize(image, to_res)))
    model.add(res_model)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(num_classes, activation='softmax'))

    # Notice that we use label_smoothing=0.1 in the loss function. 
    # When using MixUp, label smoothing is highly recommended.
    model.compile(
        loss=losses.CategoricalCrossentropy(label_smoothing=0.1),
        optimizer=optimizers.Adam(config["learning_rate"]),
        metrics=config["metrics"],
    )
    return model

In [52]:
# AUTOTUNE = tf.data.AUTOTUNE

# def configure_for_performance(ds):
#   ds = ds.cache()
#   ds = ds.shuffle(buffer_size=500)
#   ds = ds.batch(config.batch_size)
#   ds = ds.prefetch(buffer_size=AUTOTUNE)
#   return ds

# train_dataset = configure_for_performance(train_dataset)
# val_dataset = configure_for_performance(val_dataset)

In [76]:
config['epochs'] = 100
config['dry_run'] = True
config['learning_rate'] = 
os.environ['WANDB_MODE'] = 'run'

In [53]:
#!/usr/bin/env python

"""
Trains a model on the dataset.
Designed to show how to do a simple wandb integration with keras.
"""
from data_utilities import get_data
from model_utilities import get_generators, get_optimizer, get_model_weights
from models import finetune

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.random import set_seed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

from wandb.keras import WandbCallback
import wandb
import random
import numpy as np
import json
import argparse
import sys
import os
# import matplotlib.pyplot as plt

# *IMPORANT*: Have to do this line *before* importing tensorflow
# os.environ['PYTHONHASHSEED'] = str(1)


def reset_random_seeds():
    os.environ['PYTHONHASHSEED'] = str(1)
    set_seed(1)
    np.random.seed(1)
    random.seed(1)


reset_random_seeds()
args = config

if args["dry_run"]:
    wandb.mode = "offline"
else:
    wandb.mode = "online"

run = wandb.init(
    project=args["project_name"],
    notes=args["notes"],
    allow_val_change=True)

wandb.config.update(config)

print(f'Augmentation - {config["augment"]}\nmodel name - {config["model_name"]}\nconfig - {config}')

# build model

wandb.log({"num_classes": num_classes})
labels = builder.info.features['label'].names

tf.keras.backend.clear_session()
model = get_model()

# model.summary()

# model_checkpoint = ModelCheckpoint("checkpoints/"+
#                                    f"{wandb.run.name}-"+config["model_name"]+
#                                    "-epoch-{epoch}_val_accuracy-{val_acc:.2f}.hdf5", save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor="val_accuracy", factor=0.8, patience=5, verbose=1)
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=config['earlystopping_patience'], verbose=1, mode='auto',
    restore_best_weights=True
)

wandbcallback = WandbCallback(training_data=train_dataset, 
                              validation_data=val_dataset,
                              labels=labels,
                              save_model=True,
                              monitor='val_loss',
                              log_weights=True,
                              log_evaluation=True,
                              validation_steps=5)

# Define WandbCallback for experiment tracking

callbacks = [earlystopper, reduce_lr, wandbcallback]
model = get_model()


history = model.fit(
    train_dataset,
    epochs=config["epochs"],
    validation_data=val_dataset,
    callbacks = callbacks
)
# trained_model_artifact = wandb.Artifact(config["model_name"], type = "model", description=config["notes"])
# trained_model_artifact.add_dir('checkpoints')
# run.log_artifact(trained_model_artifact)

run.finish()


# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
#     model = get_model()
#     history = model.fit(
#         train_dataset,
#         epochs=3,
#         validation_data=val_dataset,
#         callbacks = callbacks
#     )


# Confution Matrix and Classification Report
# Y_pred = model.predict(
#     val_generator,
#     len(val_generator) // config["batch_size"] + 1)
# y_pred = np.argmax(Y_pred, axis=1)

# print('Confusion Matrix with Validation data')
# print(confusion_matrix(val_generator.classes, y_pred))
# print('Classification Report')
# cl_report = classification_report(
#     val_generator.classes,
#     y_pred,
#     target_names=labels,
#     output_dict=True)
# # print(pd.DataFrame(cl_report))
# print(cl_report)
# run.finish()

# Training

In [ ]:
%%writefile config.json

{
    "root_dir" : "data/1_extracted/Rock_Dataset/",
    "project_name" : "rock_classification",
    "model_name" : "efficientnet",
    "sample_size" : 0.5,
    "augment": "False",
    "optimizer": "Adam",
    "learning_rate" : 0.005,
    "batch_size" : 128,
    "epochs" : 50,
    "image_size" : 224,
    "trainable" : false
}

Writing config.json


- use dry_run if you don't want to log to wandb

In [ ]:
!python train.py --dry_run

python3: can't open file 'train.py': [Errno 2] No such file or directory


# Weights and Biases Sweeps

In [ ]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
sweep_id = '3pw3l2bd'
%cd /content/Whats-this-rock
!wandb agent rock-classifiers/Whats-this-rock/$sweep_id

/content/Whats-this-rock
wandb: Login to W&B to use the sweep agent feature
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [ ]:
!wandb sweep --update rock-classifiers/Whats-this-rock/$sweep_id sweep.yaml

Traceback (most recent call last):
  File "/usr/local/bin/wandb", line 5, in <module>
  File "/usr/local/lib/python3.7/dist-packages/wandb/__init__.py", line 26, in <module>
    from wandb import sdk as wandb_sdk
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/__init__.py", line 7, in <module>
    from .wandb_artifacts import Artifact  # noqa: F401
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_artifacts.py", line 29, in <module>
    from wandb.apis import InternalApi, PublicApi
  File "/usr/local/lib/python3.7/dist-packages/wandb/apis/__init__.py", line 31, in <module>
    from .internal import Api as InternalApi  # noqa
  File "/usr/local/lib/python3.7/dist-packages/wandb/apis/internal.py", line 1, in <module>
    from wandb.sdk.internal.internal_api import Api as InternalApi
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/internal/internal_api.py", line 33, in <module>
    from ..lib.git import GitRepo
  File "/usr/local/lib/python3.7/dist-packages/

# Deploy Telegram Bot

Uploading secrets with telegram key

In [ ]:
print("Upload secrets.json")
files.upload();